In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import combinations  # For creating combinations of elements
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit,train_test_split
import xgboost as xgb
from sklearn import svm
import catboost as cbt
import lightgbm as lgb 

# tool functions 
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    # 📏 Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    # ℹ️ Provide memory optimization information if 'verbose' is True
    if verbose:
        print(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    # 🔄 Return the DataFrame with optimized memory usage
    return df



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# input data
df_train = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
print(f"original shape {df_train.shape}")
print(df_train[0:1])
# reduce mem

original shape (5237980, 17)
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0        0                  0      3180602.69   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                        1         0.999812   13380276.64        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap    target  \
0         NaN   0.999812   60651.5   1.000026   8493.03  1.0 -3.029704   

   time_id row_id  
0        0  0_0_0  


## **data cleaning functions**

In [3]:
# drop the unneccassary columns
def record_filter(df:pd.DataFrame)->pd.DataFrame:
    if('target'in df):
        df.dropna(subset=['target'],inplace=True) # drop record target =null
        df.reset_index(drop=True, inplace=True)
    return df

def feature_selection(df:pd.DataFrame,NotIncludeAtrs=['row_id', 'time_id','date_id','stock_id'])->pd.DataFrame:
    cols = [c for c in df.columns if c not in NotIncludeAtrs]
    tmp = df[cols] 
    return tmp


## **feature engineering functions**

In [4]:
from numba import njit, prange

@njit(parallel=True) #faster 
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    #  Loop through all combinations of triplets
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        
        # Loop through rows of the DataFrame
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            # Prevent division by zero
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

def calculate_triplet_imbalance_numba(price:[], df:pd.DataFrame):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance using the Numba-optimized function
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

# feature engineering
def feature_engineering_ver1(df:pd.DataFrame):
    df = df.copy()
    #df_copy['trend']=df['imbalance_size'] * df['imbalance_buy_sell_flag']
    df["volume"] = df["ask_size"] +df["bid_size"]
    df["mid_price"] = (df['ask_price']+df['bid_price'])/2
    df["liquidity_imbalance"] = (df['bid_size']-df['ask_size'])/(df['bid_size']+df['ask_size'])
    df["matched_imbalance"] = (df['imbalance_size']-df['matched_size'])/(df['matched_size']+df['imbalance_size'])
    df["size_imbalance"] = df['bid_size'] / df['ask_size']
   
    
    # create different pariwise combination of imbalance prices 
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    for c in combinations(prices, 2):
        #df_copy[f'{c[0]}_-_{c[1]}'] = (df_copy[f'{c[0]}'] - df_copy[f'{c[1]}']).astype(np.float32) ## difference between the different prices 
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
        
    #create triplet of imbalance
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    
    
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_velocity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance'] # showing the how hit of the stock in the market
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    
    # Calculate various statistical aggregation features
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
    
    # time related feature
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3,10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window) # keep track for previous value
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window) # percentage change with previous value
    
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'market_urgency', 'imbalance_momentum', 'size_imbalance']:
        for window in [1, 2, 3,10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window) # difference with previous value

    
    # adding day for haveing trading
    df["dow"] = df["date_id"] % 5                 # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  # Seconds
    df["minute"] = df["seconds_in_bucket"] // 60  # Minutes  
    #global feature 
    global_stock_id_feats = {
        "median_size": df.groupby("stock_id")["bid_size"].median() + df.groupby("stock_id")["ask_size"].median(),
        "std_size": df.groupby("stock_id")["bid_size"].std() + df.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df.groupby("stock_id")["bid_size"].max() - df.groupby("stock_id")["bid_size"].min(),
        "median_price": df.groupby("stock_id")["bid_price"].median() + df.groupby("stock_id")["ask_price"].median(),
        "std_price": df.groupby("stock_id")["bid_price"].std() + df.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df.groupby("stock_id")["bid_price"].max() - df.groupby("stock_id")["ask_price"].min(),
    }
    for key, value in global_stock_id_feats.items():
            df[f"global_{key}"] = df["stock_id"].map(value.to_dict())
    
    df.replace([np.inf, -np.inf], 0,inplace=True)
    return df

## **train data**

In [5]:
def preproccess(df:pd.DataFrame,NotIncludeAtrs=['row_id', 'time_id','date_id','stock_id']):
    df=record_filter(df) 
    df=feature_engineering_ver1(df)
    df=feature_selection(df,NotIncludeAtrs)
    df=reduce_mem_usage(df)
    df=df.fillna(0)
    return df
df_train=preproccess(df_train,NotIncludeAtrs=['row_id', 'time_id','stock_id'])

/opt/conda/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'comb_indices' of function 'compute_triplet_imbalance'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../tmp/ipykernel_27/1084247445.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/opt/conda/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'comb__indices' of function '__numba_parfor_gufunc_0x78e0b7fb9e70'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<string>", lin

In [6]:
models=[]
def trainEnsembleModels(folds=5,data:pd.DataFrame=None):
    models=[]
    data=data.copy()
    max_date=data['date_id'].max()
    folds=folds-1
    date_bins = pd.cut(data['date_id'], bins=folds, labels=False)
    for fold in range(folds):
        
        if fold >= folds-1:
            subData = data.loc[(date_bins == fold)] #block
            date = subData['date_id'].max()-3 # use 3 date for validation
            trainingData=subData[subData['date_id']<=date]
            validData=subData[subData['date_id']>date]
            del subData
        else:
            trainingData = data.loc[(date_bins == fold)] #block
            temp = data.loc[(date_bins >= (fold+1))]
            minDate = temp["date_id"].min()
            validData=temp[temp["date_id"]<(minDate+3)]
        
        trainingData=trainingData.drop(columns='date_id')# give date_id
        validData=validData.drop(columns='date_id')
        
        x_train=trainingData.drop(columns='target')
        y_train=trainingData['target'].values
        
        x_valid=validData.drop(columns='target')
        y_valid=validData['target'].values
        
        print(f"train.shape {x_train.shape} valid.shape {x_valid.shape}")
        #train model
        model = lgb.LGBMRegressor(n_estimators= 5500,objective='mae',learning_rate=0.00871,num_leaves=256,subsample=0.6,colsample_bytree=0.8,max_depth=11,importance_type= "gain",verbosity= -1,device="gpu",seed= 42)
        model.fit(x_train,y_train,
                  eval_set=[(x_valid,  y_valid)],
                  callbacks=[
                        lgb.callback.early_stopping(stopping_rounds=100),
                        lgb.callback.log_evaluation(period=100),
                    ],
                 )
        models.append(model)
        print(f"append model of fold {fold}")
        del x_train,y_train,x_valid,y_valid,trainingData,validData
    
    #train all data again
    average_best_iteration = int(np.mean([model.best_iteration_ for model in models]))
    print(f"average_best_iteration {average_best_iteration}")
    all_y_train = data['target'].values
    all_x_train = data.drop(columns=['target','date_id'])
    print(f"all train.shape {all_x_train.shape}")
    model = lgb.LGBMRegressor(n_estimators= average_best_iteration,objective='mae',learning_rate=0.00871,num_leaves=256,subsample=0.6,colsample_bytree=0.8,max_depth=11,importance_type= "gain",verbosity= -1,device="gpu",seed= 42)
    model.fit(all_x_train,all_y_train,
                callbacks=[
                    lgb.callback.log_evaluation(period=100),
                ],
            )
    models.append(model) # total is k models + 1 final model
    return models

y_min = df_train['target'].min()
y_max = df_train['target'].min()
models=trainEnsembleModels(folds=5,data=df_train)

train.shape (1294779, 123) valid.shape (32505, 123)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 9.72964
[200]	valid_0's l1: 9.66512
[300]	valid_0's l1: 9.63859
[400]	valid_0's l1: 9.62467
[500]	valid_0's l1: 9.61674
[600]	valid_0's l1: 9.61186
[700]	valid_0's l1: 9.61053
[800]	valid_0's l1: 9.60946
[900]	valid_0's l1: 9.60935
[1000]	valid_0's l1: 9.60769
[1100]	valid_0's l1: 9.60723
Early stopping, best iteration is:
[1031]	valid_0's l1: 9.60717
append model of fold 0
train.shape (1306195, 123) valid.shape (32835, 123)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 5.91934
[200]	valid_0's l1: 5.90024
[300]	valid_0's l1: 5.89266
[400]	valid_0's l1: 5.88686
[500]	valid_0's l1: 5.88536
[600]	valid_0's l1: 5.88401
[700]	valid_0's l1: 5.88385
[800]	valid_0's l1: 5.88318
[900]	valid_0's l1: 5.88313
[1000]	valid_0's l1: 5.88306
[1100]	valid_0's l1: 5.88321
Early stopping, best iteration is:
[1033]	valid_0's l1: 5.88266
a

In [7]:
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)  # 🧮 Calculate standard error based on volumes
    step = np.sum(prices) / np.sum(std_error)  # 🧮 Calculate the step size based on prices and standard error
    out = prices - std_error * step  # 💰 Adjust prices by subtracting the standardized step size
    return out

def zero_sum_prediction(predY,data):
    predY = zero_sum(predY, data['bid_size'] + data['ask_size'])
    clipped_predictions = np.clip(predY, -64, 64)
    return clipped_predictions


In [8]:
import optiver2023
optiver2023.make_env.func_dict['__called__'] = False
env = optiver2023.make_env()
iter_test = env.iter_test()


cache = pd.DataFrame()
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if("currently_scored" in test):
        test=test.drop("currently_scored",axis=1)
        
    cache = pd.concat([cache, test], ignore_index=True, axis=0)
    if counter > 0: # take 4 week as windows
        cache = cache.groupby(['stock_id']).tail(28).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
        
    # 📊 Generate features based on the updated cache
    print(f"------counter :{counter}-----------")
    print(cache.shape)
    print(test.shape)
    test_df=preproccess(cache)# pick the last 200 record need to be predict
    test_df=test_df[-test.shape[0]:]
    pred = np.mean([model.predict(test_df) for model in models], axis=0)
    pred= zero_sum_prediction(pred,test)
    print(pred.values)
    sample_prediction['target'] = pred
        
    env.predict(sample_prediction)
    
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
------counter :0-----------
(200, 15)
(200, 15)


/opt/conda/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'comb_indices' of function 'compute_triplet_imbalance'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../tmp/ipykernel_27/1084247445.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/opt/conda/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'comb__indices' of function '__numba_parfor_gufunc_0x78e00ae62e30'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<string>", lin

[-0.49928786  2.00450398  2.88617441 -0.59328987 -0.66364197  0.1828317
  1.30624319 -3.34288366 -2.36343669  1.78506156 -0.02583075 -3.58476951
 -0.94310311 -1.25062005 -0.62238419 -0.09801277 -0.86525588  1.73308589
  1.48477888  5.23814711  0.62098899  0.10947733 -1.13797233 -4.22912334
 -0.33638737  1.5063893  -1.7283274  -0.18798193  0.54385447  1.73738263
 -0.04317951 -5.68021804 -2.42942958 -0.50566502  4.21138843 -0.74024154
 -0.31778277 -0.03067026 -1.04863765  2.80367027 -0.52778536  0.73107198
 -0.74034224  2.07644734 -0.32684344 -1.06747228 -1.17328332  3.32951188
  1.93800908 -0.88845477 -3.50788192 -3.89925894  0.01486392 -0.62506447
  1.57647503 -0.40922506 -0.06405479 -0.07172021  0.44823519  0.08227457
 -0.79112605  1.5439489   3.71076854  0.2026549  -1.57207041 -0.97918041
 -0.0628934   0.37694566 -0.08421949 -1.59053764  3.30382052  2.09276285
 -1.44339335 -0.93300569  0.42432437  2.39226771 -1.16911336  3.49052045
 -1.72421997 -1.25666359 -1.96595292 -0.07889635 -0.

/tmp/ipykernel_27/1084247445.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window) # difference with previous value
/tmp/ipykernel_27/1084247445.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window) # difference with previous value
/tmp/ipykernel_27/1084247445.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

[ 2.46947619e-01  1.73415760e+00  5.43119583e-01 -1.85162201e-01
 -4.79230212e-01 -2.61425034e-04 -3.51629093e-01  1.10155436e+00
  2.29539006e+00  1.00168453e+00 -5.14552255e-01 -6.41950009e-01
 -2.68853563e-01 -1.46695441e-01  1.07798022e+00 -9.35484650e-01
 -2.54621431e+00  2.75530788e+00 -8.94972296e-01  4.46036394e+00
  7.51617801e-01 -7.01840422e-02 -6.27016635e-02 -6.42565211e-01
 -4.00359910e-01  1.09744465e+00 -1.13580012e+00  1.20041129e+00
 -9.36181334e-01  1.68974475e+00  2.63006477e-02 -4.89723597e+00
  1.04602841e-01 -1.04045412e+00 -2.31527497e+00  5.18027667e-02
 -5.69884164e-01  3.78593501e-01 -2.45405562e-01  1.46179070e-01
 -8.50270582e-01 -4.96969889e+00 -6.73631771e-01  2.91049923e+00
  4.56251730e-01 -4.54363080e-01 -3.05767895e-01  1.22138996e+00
 -7.90505313e-01 -1.04838507e+00  3.64079151e-01 -1.05265164e+00
  5.53796753e-01 -2.64988526e-01  2.77234477e+00 -4.04150399e-01
  1.17639305e-01 -4.61621125e+00  2.90352522e-01  4.92076959e-01
 -1.29602193e-01  9.75844

...

[-0.75081607  3.0878883   0.84586579 -0.42945142 -0.25117837  0.65032142
 -0.35849784 -2.00102131 -0.47503147 -0.67699076  0.3134395   0.43351005
 -0.12860656  0.42754354 -1.17422251 -1.10152215 -1.2843881   0.89044104
  0.2424181  -0.83288146 -0.2795779   0.11764752 -0.99845257  0.3806903
 -0.07518529  0.35131511  0.57429176  0.1934501  -0.76436983  0.40994982
  1.06361945  0.64871332 -0.40027759 -0.95047553 -0.19739121  0.08923029
 -0.06194971 -0.16029947 -0.00380567  0.08815599 -0.52755831 -0.45895893
 -0.36515995 -0.00790521 -0.41954938 -1.31097144 -0.57622984  0.40721304
  1.01855404  0.2607885   0.08936884 -0.11802633 -0.33315216  0.4814986
  1.47480318 -0.80963917  0.9950114  -0.99216814  0.71510269  0.74220175
  0.89089875 -0.35589728  2.29371504  0.29908099  0.8407476  -0.03112248
 -0.2148797  -1.52522987 -0.1681505  -1.20404515 -0.20858068  0.29449687
 -0.17801788  0.25330963 -0.09942342 -1.04239163 -0.44983999 -0.66855358
  0.18604406  0.49883915 -0.27381221 -0.01474329  1.2

/tmp/ipykernel_27/1084247445.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window) # difference with previous value
/tmp/ipykernel_27/1084247445.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window) # difference with previous value
/tmp/ipykernel_27/1084247445.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

[-5.12740952e-01  2.34460754e+00  7.87359060e-01 -5.71276427e-01
 -7.61374897e-01  3.00438031e-02  1.48076010e+00 -2.12142367e-01
  5.20215273e-01 -1.57668347e-01  1.04144404e+00  5.19818933e-01
 -1.02495684e-01  8.84157284e-01 -5.14795787e-01 -7.42757188e-01
  7.57762212e-01  7.40140014e-01  1.37510670e-01  1.69862655e+00
 -3.15616366e-01  1.22856241e-01 -9.67651160e-01 -4.52550566e-01
 -2.78247274e-01 -1.19894794e-01  8.40806507e-01  2.12045672e-02
 -1.11373166e+00  1.46567245e+00  7.42884950e-01  1.14555222e+00
 -6.49429211e-01  1.35995975e-01 -1.17131403e-01  8.67496422e-02
  2.60379683e-01 -1.22160476e-01 -6.06345355e-03  1.09789124e-01
 -4.57845796e-01 -4.61409505e-01 -8.79403286e-02 -3.14205393e-01
  3.67692387e-01 -1.23490282e+00  1.49182568e-01  3.05703502e-01
  2.90617165e-01  1.62902406e-01 -2.12086007e-01  9.78852387e-02
  3.78921961e-01  5.71689000e-01  6.81605543e-01 -8.35034383e-01
  1.79905703e-01  2.11069637e-01  2.09963853e-01  6.82064132e-01
  2.32167850e-01 -4.28404

/tmp/ipykernel_27/1084247445.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window) # difference with previous value
/tmp/ipykernel_27/1084247445.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window) # difference with previous value
/tmp/ipykernel_27/1084247445.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

[-1.0754131   0.28423224  0.56022283 -1.488577   -1.03084942  2.36151117
  1.63087166 -0.42779804  0.72866057 -0.69206039  1.34438517  2.5688852
  0.47815856  2.5816572  -1.98064451 -0.76928713 -2.28310459  0.50104679
  2.01061605 -0.07628511 -0.40374853 -1.620792   -1.41066596  0.9987472
  0.95280079 -0.89994487  5.88990427 -0.04825404 -3.10843349  2.43189525
  1.48225174  2.92961711 -1.46470794  1.1952923   0.12893416 -0.30129495
 -0.30572223 -0.25473728 -0.56606034  0.18198697  1.49748374 -0.49034241
  0.07332062 -0.35959608  1.26916133 -2.28349719 -1.23361792 -0.85994674
  0.96418431  0.97271115 -1.6730443  -0.20687132 -0.2884918   0.97490414
 -0.26333715 -2.37185663  0.3610724  -0.11611046  1.05161372  0.87722085
  0.53048491  2.54447593 -0.05388463  0.13572571 -0.83089679  0.79187001
 -0.96780368 -1.66486597 -0.50549644 -8.96248455 -0.03576052  0.92071767
 -0.05758293  2.04481738  2.61709688 -2.02071284 -0.03864915 -1.86155165
 -1.66945611 -1.70256994 -0.64483537  0.77460515 12.6

In [9]:
print("done")

done
